# Clase 4: Errores de redondeo

Basado en la sección 1.2 del libro de Burden.

El computador representa los números usando un sistema **binario**. Es decir, una lista de 1s y 0s. Las CPUs modernas usan 64 bits, cada uno de los cuales puede ser 0 o 1. Como esto es un número finito de información, la precisión del computador para guardar un número no es infinita y esto puede inducir errores.

Para entender el problema, hagamos un ejemplo con 16 bits.

In [3]:
import numpy as np

In [18]:
pi = np.float16(np.pi)
dos = np.float16(2)
cien = np.float16(100)
cien*np.sin(dos*pi)

-0.1935

## Representación de los números en un sistema binario

El computador usa 64 bits para representar un número decimal. Por ejemplo

1 11011011111 01100000001010100000000000000000000000000000000000000

El primer bit se llama $s$, los siguientes 10 bits se llaman $c$. Los siguientes 52 bits forman una fracción que consiste en la siguiente suma

$$
f = \sum_{i = 1}^{52}\left(\frac{b_i}{2}\right)^i\,,
$$

donde $b_i$ es el $i$ bit de esos 52. El número total está dado por

$$
(-1)^s 2^{c - 1023} (1 + f)\,.
$$

La fracción de nuestro ejemplo es

$$
f = \left(\frac{1}{2}\right)^2 + \left(\frac{1}{2}\right)^3 + \left(\frac{1}{2}\right)^{11} +  \left(\frac{1}{2}\right)^{13} + \left(\frac{1}{2}\right)^{15} = 0.375640869140625
$$

El número $c$ está escrito en binario, lo que quiere decir

$$
c = \sum_{i = 0}^{10} b_i 2^{10 - i}
$$

En nuestro ejemplo esto es

$$
c = 2^{10} + 2^9 + 2^7 + 2^6 + 2^4 + 2^3 + 2^2 + 2^1 + 2^0 = 1759
$$

Entonces el número escrito arriba corresponde a

$$
(-1)\times 2^{1759 - 1023} 1.375640869140625 \approx -4.97258\times 10^{221}
$$

Hay dos valores de $c$ con significado especial: Todos 0 representa que en la mantisa se toma $2^{-1022}f$ en vez de $2^{-1023}(1 + f)$. Todos $1$ representa $\infty$.

* Número más grande de 64 bits (todos 1 excepto $s$ y el último dígito de $c$)

0 11111111110 1111111111111111111111111111111111111111111111111111

In [239]:
f = sum((1/2.)**(np.arange(1,53)))
f

0.9999999999999998

In [274]:
c = sum(2**(np.arange(1,11)))
c - 1023

1023

In [277]:
2**1023*(1+f)

1.796815355011309e+308

* Número más pequeño de 64 bits (todos 1 excepto el último dígito de c)

1 11111111110 1111111111111111111111111111111111111111111111111111

In [279]:
-2**1023*(1+f)

-1.796815355011309e+308

* Número de 64 bits más cercano a cero (todos cero excepto el último dígito de $c$)

0 00000000001 0000000000000000000000000000000000000000000000000000

In [280]:
c  =  1
c - 1023

-1022

In [281]:
2**(-1022)

2.2250738585072014e-308

* Cifras decimales de precisión de 64 bits (todos cero excepto el último dígito de $f$)

0 00000000000 0000000000000000000000000000000000000000000000000001

In [282]:
(1/2)**(52)

2.220446049250313e-16

### Números de 16 bits

Para ilustrar el tipo de dificultades que esto introduce es engorroso trabajar con números tan grandes. Por eso trabajamos con números de 16 bits (como si estuviéramos en los años 90). Un número de este estilo es

1 11010 1111000100

El primer dígito es $s$, los siguientes cinco forman $c$ y los siguientes diez forman $f$ de manera análoga a la de antes. En nuestro ejemplo tenemos

$$
f = \left(\frac{1}{2}\right)^1 + \left(\frac{1}{2}\right)^2 + \left(\frac{1}{2}\right)^3 + \left(\frac{1}{2}\right)^4 + \left(\frac{1}{2}\right)^8 = 0.94140625
$$

$$
c = 2^4 + 2^3 + 2^1 = 26
$$

El número total está dado por $$(-1)^s 2^{c - 15} (1 + f)$$

El número de nuestro ejemplo es

$$
(-1)\times 2^{26 - 15} 1.94140625 = -3976
$$

* Número más grande de 16 bits

0 11110 1111111111

In [243]:
f = sum((1/2.)**(np.arange(1,11)))
f

0.9990234375

In [286]:
c = sum(2**(np.arange(1,5)))
c

30

In [287]:
2**(c - 15)*(1 + f)

65504.0

* Número más pequeño de 16 bits

1 11110 1111111111

In [294]:
-2**(c-15)*(1 + f)

-65504.0

* Número más cercano a cero de 16 bits

0 00000 0000000001

In [296]:
2**(1-15)*((1/2)**10)

5.960464477539063e-08

* Cifras decimales de precisión de 16 bits

0 00000 0000000001

In [291]:
(1/2)**(10)

0.0009765625

## Redondeo, truncación, error absoluto y relativo

Para explorar este problema, ignoremos por ahora el sistema *binario* y veamos lo que ocurre en el sistema **decimal**. 

In [19]:
np.pi

3.141592653589793

En este caso, si queremos representar el número $\pi$ con una cierta cantidad $n$ de cifras significativas podemos hacer dos cosas:

* Truncar: Ignoramos las cifras adicionales, más allá de la $n$-ésima cifra.

In [20]:
pi_truncado = 3.1415

* Redondear: Si la cifra $n + 1$ es menor a 5 truncamos, pero si la cifra $n + 1$ es mayor o igual que 5

In [21]:
pi_redondeado = 3.1416

Al truncar o redondear cometemos un error. Podemos cuantificar el error de varias maneras:

* Error real: estimación - valor verdadero

In [22]:
pi_truncado - np.pi

-9.265358979293481e-05

In [23]:
pi_redondeado - np.pi

7.346410206832132e-06

* Error absoluto: |estimación - valor verdadero|

In [25]:
np.abs(pi_truncado - np.pi)

9.265358979293481e-05

In [26]:
np.abs(pi_redondeado - np.pi)

7.346410206832132e-06

* Error relativo: |estimación - valor verdadero|/|valor verdadero|

In [27]:
np.abs(pi_truncado - np.pi)/np.abs(np.pi)

2.9492553621508708e-05

In [28]:
np.abs(pi_redondeado - np.pi)/np.abs(np.pi)

2.3384349967961744e-06

## Aritmética de dígitos finitos

Lo que hace el computador es aproximar el resultado de cada operación haciendo una truncación o redondeo. Esto quiere decir que si quiero multiplicar el número $2$ por el número $\pi$, primero aproximo $2 \simeq fl(2)$ y $\pi \simeq fl(\pi)$ (donde $fl$ representa la aproximación de un número dado), y luego de hacer la multiplicación vuelve a aproximar $$2\pi \simeq fl(fl(2)\times fl(\pi))$$.

**Hagamos un ejemplo.** Supogamos que queremos calcular lo siguiente

$$10000\times(u - v)$$

donde $u = 1/5$ y $v = 399/2000$. Sabemos que el resultado debe ser $5$. Veamos qué pasa si usamos números de 16 bits.

In [38]:
fl_u = np.float16(1)/np.float16(5)
fl_v = np.float16(399)/np.float16(2000)
np.float16(10000)*(fl_u - fl_v)

4.883

In [41]:
np.abs(np.float16(10000)*(fl_u - fl_v) - 5)/5

0.0234375

Esto ocurre porque hay una resta entre dos números muy similares, que luego se multiplica por un número grande. Este tipo de errores ocurre también para los números que usamos de 64 bits, para números más cercanos multiplicados por números más grandes.

**Hagamos otro ejemplo.** Busquemos la solución de la ecuación: $$x^2 + 200 x + 1 = 0$$

Las soluciones son $$x = \frac{-200 \pm \sqrt{200^2 - 4}}{2}$$

In [56]:
doscientos = np.float16(200)
dos = np.float16(2)
cuatro = np.float16(4)
discr = np.float16(np.sqrt(np.float16(doscientos**2) - cuatro))
x1 = np.float16(np.float16(-1*doscientos - discr)/dos)
x2 = np.float16(np.float16(-1*doscientos + discr)/dos)

In [57]:
x1

-200.0

In [58]:
x2

0.0

In [65]:
x1v = (-200 - np.sqrt(200**2 - 4))/2
x2v = (-200 + np.sqrt(200**2 - 4))/2

In [66]:
x1v

-199.99499987499377

In [67]:
x2v

-0.005000125006247913

In [ ]:
x1 - x1v

Una manera de resolver este problema, es escribir la solución 2 para que no contenga la diferencia de dos números muy similares, esto se logra escribiendo

$$
\frac{-200 + \sqrt{200^2 - 4}}{2} = \frac{(-200 + \sqrt{200^2 - 4})}{2}\frac{(-200 - \sqrt{200^2 - 4})}{(-200 - \sqrt{200^2 - 4})} = \frac{4}{2(-200 - \sqrt{200^2 - 4})}
$$

In [70]:
np.float16(dos/np.float16(-1*doscientos - discr))

-0.005

En general cuando estamos tratando de resolver una ecuación con coeficientes que no tienen el mismo orden de magnitud encontraremos este tipo de problemas.

**Otro ejemplo**. Consideremos que queremos evaluar la siguiente función

$$
P(x) = x^3 - 6.1 x^2 + 3.2 x + 1.5
$$

en el punto $x = \pi$.

In [199]:
tres = np.float16(3)
t1 = np.float16(pi**tres)
t2 = np.float16(np.float16(6.1)*np.float16(pi**dos))
t3 = np.float16(np.float16(3.2)*pi)
t4 = np.float16(1.5)
aproximado = np.float16(np.float16(np.float16(t1 - t2) + t3) + t4)
exacto = np.pi**3 - 6.1*np.pi**2 + 3.2*np.pi + 1.5

In [200]:
aproximado

-17.69

In [201]:
exacto

-17.645213674857924

In [202]:
np.abs(aproximado - exacto)/np.abs(exacto)

0.002396475663104501

Este error es relativamente grande porque la expresión original contiene un gran número de operaciones y cada operación introduce un error. Podemos reducir este error reduciendo el número de operaciones que tenemos que hacer. Para eso reescribimos la expresión de la siguiente manera

$$P(x) = x(x(x - 6.1) + 3.2) + 1.5$$

Esta se llama aritmética anidada y reduce el número de operaciones.

In [205]:
p1 = np.float16(pi - np.float16(6.1))
p2 = np.float(np.float16(pi*p1) + np.float16(3.2))
aproximado = np.float16(np.float16(pi*p2) + np.float16(1.5))
exacto = np.pi**3 - 6.1*np.pi**2 + 3.2*np.pi + 1.5

In [206]:
np.abs(aproximado - exacto)/np.abs(exacto)

0.0006254571548658005

## Tareas

### Tarea 2.6

Escriba a cuál número corresponde la siguiente representación de 16 bits

1 00110 1110110101

**Solución:** $$-2^{6-15}\left[\left(\frac{1}{2}\right) + \left(\frac{1}{2}\right)^2 + \left(\frac{1}{2}\right)^3 + \left(\frac{1}{2}\right)^5 + \left(\frac{1}{2}\right)^6 + \left(\frac{1}{2}\right)^8 + \left(\frac{1}{2}\right)^{10}\right]$$

In [1]:
-2**(6-15)*(1 + 1/2 + (1/2)**2 + (1/2)**3 + (1/2)**5 + (1/2)**6  + (1/2)**8 + (1/2)**10)

-0.0037631988525390625

### Tarea 2.7

Escriba el siguiente número en su representación de 16 bits: 9.75

**Solución:** Como es positivo $s = 0$. Para obtener la potencia de dos que va en el exponencial sacamos el logaritmo base 2

In [21]:
np.log(9.75)/np.log(2)

3.2854022188622487

Como la potencia es $3$ tenemos que $c = 3 + 15 = 18$ que en binario se representa 10010. Para encontrar la matisa dividimos por esta potencia

In [22]:
9.75/2**3

1.21875

De modo que $f = 0.21875$. Este número es menor que $1/4 = 0.25$ pero mayor que $1/8 = 0.125$, por lo que sabemos que el primer y segundo dígitos de la mantisa deben ser cero y tenemos $f = 001...$

Esto lo podemos confirmar tomando el logaritmo en base 2

In [23]:
np.log(9.75/2**3 - 1)/np.log(2)

-2.1926450779423963

Ahora le restamos $1/8$ y tomamos el logaritmo para ver cuál es la siguiente potencia

In [24]:
np.log(9.75/2**3 - 1 - 1/8)/np.log(2)

-3.415037499278844

In [25]:
np.log(9.75/2**3 - 1 - 1/8 - 1/16)/np.log(2)

-5.0

Como este logaritmo es exacto, tenemos que el resultado es

In [27]:
1 + 1/2**3 + 1/2**4 + 1/2**5

1.21875

Entonces la mantisa es 0011100000

El número es 0 10010 0011100000

### Tarea 2.8

Use la serie de Taylor del seno para calcular $\sin(2)$ con un error relativo menor que $0.01$.

In [2]:
import numpy as np

In [6]:
suma = 0
error = 1
i = 0
while error > 0.01:
    suma = suma + (1/np.math.factorial(2*i+1))*(-1)**i*2**(2*i + 1)
    i += 1
    error = np.abs(suma - np.sin(2))/np.abs(np.sin(2))
print('aproximacion = ', suma, ', error = ', error)

aproximacion =  0.9079365079365079 , error =  0.0014966707801260235


In [35]:
aprox

0.9079365079365079

In [36]:
np.sin(2)

0.9092974268256817

Suponga que queremos calcular $e^{-2.4}$ y lo aproximamos como $1 - 2.4 + \frac{1}{2}\times (2.4)^2 - \frac{1}{6}\times (2.4)^3 + \frac{1}{24} (2.4)^4 - \frac{1}{120}(2.4)^5 + \frac{1}{720}(2.4)^6$. Calcule el error absoluto y relativo. ¿Es una buena aproximación?

In [112]:
aprox = 1 - 2.4 + (1/2)*2.4**2 - (1/6)*2.4**3 + (1/24)*2.4**4 - (1/120)*2.4**5 + (1/720)*2.4**6
abs_error = np.abs(aprox - np.exp(-2.4))
rel_error = np.abs(aprox - np.exp(-2.4))/np.abs(np.exp(-2.4))

In [113]:
abs_error

0.06955084671058809

In [114]:
rel_error

0.7666712507137792

A pesar de que el error absoluto es relativamente pequeño, del error relativo vemos que no es una buena aproximación. El error absoluto es pequeño en este caso porque el valor de la función es en sí mismo pequeño.

In [115]:
np.exp(-2.4)

0.09071795328941251

In [116]:
aprox

0.1602688000000006

### Tarea 2.9

Ejercicio 22, sección 1.2 del libro de Burden. 

Queremos aproximar la función $e^{-5}$ y lo hacemos de dos maneras

* $\sum_{n=0}^9 \frac{1}{n!}(-5)^n$


* $\frac{1}{\sum_{n=0}^9 \frac{1}{n!} 5^n}$

Calcule los errores absolutos y relativos de cada aproximación. ¿Cuál es más precisa y por qué?

In [310]:
aprox1 = 0
aprox2_denom = 0
for n in range(10):
    aprox1 += (1/np.math.factorial(n))*(-5)**n
    aprox2_denom += (1/np.math.factorial(n))*5**n
aprox2 = 1/aprox2_denom

abs_err1 = np.abs(aprox1 - np.exp(-5))
abs_err2 = np.abs(aprox2 - np.exp(-5))
rel_err1 = np.abs(aprox1 - np.exp(-5))/np.exp(-5)
rel_err2 = np.abs(aprox2 - np.exp(-5))/np.exp(-5)

In [311]:
abs_err1

1.8338433261878

In [312]:
abs_err2

0.00022150586456406995

In [313]:
rel_err1

272.16648133870825

In [314]:
rel_err2

0.0328743851197011

La primera aproximación es mala porque contiene varios términos alternantes que deben alternarse para obtener un número pequeño. Esa cancelación no es exacta e introduce errores numéricos.

### Tarea 2.10

En ciertos cálculos importantes en cosmología surge un problema análogo al siguiente. Queremos calcular algunas integrales

$$
I_{1} = \int_{0}^{q_{max}} q^2\left(\frac{1}{q^4} + \frac{3}{q}\right)\,dq
$$

$$
I_{2} = \int_{0}^{q_{max}} q^2\left(\frac{-1}{q^4} + \frac{1}{q}\right)\,dq
$$

Estas integrales se pueden aproximar numéricamente por medio de su suma

$$
\int_0^{q_{max}} f(q)\,dq \approx \frac{q_{max}}{N} \sum_{i = 1}^{N} f\left(i*\frac{q_{max}}{N}\right) 
$$

Profundizaremos en el cálculo numérico de integrales más adelante.

La cantidad de interés es $I = I_{1} + I_{2}$.

* Usando $q_{max} = 0.1$ y $N = 500000$, calcule ambas integrales por separado usando la aproximación, y luego súmelas.

In [337]:
sum1 = 0
sum2 = 0
qmax =  0.1
n = 500000
for i in range(1,n+1):
    qi = i*qmax/n
    sum1 += 1/qi**3 + 3*qi
    sum2 += -1/qi**3 + qi

aprox1 = sum1*qmax/n
aprox2 = sum2*qmax/n
res = aprox1 + aprox2

In [338]:
res

0.0234375

* La suma de las integrales $I$ se puede escribir como la integral de la suma de los integrandos:

$$
I = \int_0^{q_{max}} q^2 \frac{4}{q}\,dq
$$

Calcule $I$ de esta manera usando la aproximación.

In [345]:
suma = 0
qmax =  0.1
n = 500000
for i in range(1,n+1):
    qi = (i/n)*qmax
    suma += 4*qi

res = suma*qmax/n

In [346]:
res

0.02000004

* ¿Por qué son diferentes los resultados? Compare con el resultado exacto de la integral $I$.

Al resolver la integral $I$ resultado exacto es

$$
I = 2q_{max}^2
$$

que nos da $I = 0.02$.

El primer resultado tiene un error mayor ya que involucra cancelaciones del término que va como $1/q^3$ en el integrando. Para $q$ pequeño ese término es muy grande y domina el resultado. Se pierde precisión al cancelarlo entre las dos integrales.